In [1]:
import cv2
import time
import numpy as np
import os
import json

class GenTrainingJson(object):
    
    def generateSingleAnnotation(self, fileName, annotation):
        with open(fileName, 'w') as f:
            json.dump({"data": annotation}, f)
#         print('created: ' + fileName)
        
    def readAnnotation(self, fileName):
        with open(fileName) as f:
            annotation = json.load(f)['data']
        return annotation

    def generateJson(self, dir_label):  #[{dir: 'warriorPoseI/', label: 'WarriorPoseI'}, {...}, ... ]
        PoseNet = {"data": []}
        OpenPose = {"data": []}
        for obj in dir_label:
            poseNetDic = {}
            openPoseDic = {}
            i = 0
            print("looping through: " + obj['dir'] + ' directory')
            for filename in os.scandir(obj['dir']):
                PoseNetAnno = os.path.splitext(filename.path)[0]+ "_PN_annotation.json"
                OpenPoseaAnno = os.path.splitext(filename.path)[0]+ "_OP_annotation.json"

                if os.path.exists(PoseNetAnno) and os.path.isfile(PoseNetAnno) and os.path.exists(OpenPoseaAnno) and os.path.isfile(OpenPoseaAnno) and os.path.exists(filename.path):
#                     print(PoseNetAnno + ' and '+ OpenPoseaAnno + ' exist')
                    poseNetDic = self.readAnnotation(PoseNetAnno)
                    openPoseDic = self.readAnnotation(OpenPoseaAnno)
        
                    OpenPose['data'].append(openPoseDic)
                    PoseNet['data'].append(poseNetDic)
                    i += 1
                    print('image: ' + str(i))
                    
                elif (filename.path.endswith(".jpg") or filename.path.endswith(".png")) and filename.is_file():
                    posenetKps, openposekps = self.predictImageKeypoints(filename.path)
                    poseNetDic['xs'] = {i:posenetKps[i] for i in range(0, len(posenetKps) ) }
                    poseNetDic['ys'] = {"0":obj['label']}

                    openPoseDic['xs'] = {i:openposekps[i] for i in range(0, len(openposekps) ) }
                    openPoseDic['ys'] = {"0":obj['label']}

                    self.generateSingleAnnotation(PoseNetAnno, poseNetDic)
                    self.generateSingleAnnotation(OpenPoseaAnno, openPoseDic)
                        
                    OpenPose['data'].append(openPoseDic)
                    PoseNet['data'].append(poseNetDic)
                    i += 1
                    print('image: ' + str(i))
                
                
        return PoseNet, OpenPose
                  
    def swapIndices(self, kps):
        seq = [0,15,14,17,16,5,2,6,3,7,4,11,8,12,9,13,10] #reorder the index for posenet
        arr = []
        for i in seq:
            arr.append(kps[i])
        return arr
        
    def predictImageKeypoints(self, img_path):
        protoFile = "pose/coco/pose_deploy_linevec.prototxt"
        weightsFile = "pose/coco/pose_iter_440000.caffemodel"
        nPoints = 18
        POSE_PAIRS = [[1,0],[1,2],[1,5],[2,3],[3,4],[5,6],[6,7],[1,8],[8,9],[9,10],[1,11],[11,12],[12,13],[0,14],[0,15],[14,16],[15,17]]
            
        frame = cv2.imread(img_path)
        frameCopy = np.copy(frame)
        frameWidth = frame.shape[1]
        frameHeight = frame.shape[0]
        threshold = 0.1

        net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)

        t = time.time()
        # input image dimensions for the network
        inWidth = 640
        inHeight = 480
        inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight),
                                  (0, 0, 0), swapRB=False, crop=False)

        net.setInput(inpBlob)
        output = net.forward()
        print("time taken by network : {:.3f}".format(time.time() - t))

        H = output.shape[2]
        W = output.shape[3]

        # Empty list to store the detected keypoints
        points = []

        for i in range(nPoints):
            # confidence map of corresponding body's part.
            probMap = output[0, i, :, :]

            # Find global maxima of the probMap.
            minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

            # Scale the point to fit on the original image
            x = (frameWidth * point[0]) / W
            y = (frameHeight * point[1]) / H

            if prob > threshold : 
                cv2.circle(frameCopy, (int(x), int(y)), 8, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
                cv2.putText(frameCopy, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, lineType=cv2.LINE_AA)

                # Add the point to the list if the probability is greater than the threshold
                points.append((float(x), float(y)))
            else :
                points.append((0,0))
        
        posnetPoints = self.swapIndices(points)
        openPosePoints = points
        
        posnetPoints = [element for tupl in posnetPoints for element in tupl]
        openPosePoint = [element for tupl in openPosePoints for element in tupl]
        return posnetPoints, openPosePoint

In [2]:
directoryList = [
    {
        'dir': 'warriorPoseI/',
        'label': 'WarriorPoseI'
    },
    {
        'dir': 'warriorPoseII/',
        'label': 'WarriorPoseII'
    },    
    {
        'dir': 'warriorPoseIII/',
        'label': 'WarriorPoseIII'
    },
    {
        'dir': 'treePose/',
        'label': 'TreePose'
    }
]

generator = GenTrainingJson()
PoseNet, Openpose = generator.generateJson(directoryList)

with open('PoseNetV3.json', 'w') as fp:
    json.dump(PoseNet, fp)
    
with open('OpenPoseV3.json', 'w') as fp:
    json.dump(Openpose, fp)

looping through: warriorPoseI/ directory
time taken by network : 4.783
image: 1
time taken by network : 4.542
image: 2
time taken by network : 4.647
image: 3
time taken by network : 4.756
image: 4
time taken by network : 4.751
image: 5
time taken by network : 4.603
image: 6
time taken by network : 4.648
image: 7
time taken by network : 4.633
image: 8
time taken by network : 4.769
image: 9
time taken by network : 4.849
image: 10
time taken by network : 4.704
image: 11
time taken by network : 4.688
image: 12
time taken by network : 4.600
image: 13
time taken by network : 4.746
image: 14
time taken by network : 4.748
image: 15
time taken by network : 4.667
image: 16
time taken by network : 4.742
image: 17
time taken by network : 4.596
image: 18
time taken by network : 4.747
image: 19
time taken by network : 4.830
image: 20
time taken by network : 4.731
image: 21
time taken by network : 4.741
image: 22
time taken by network : 4.580
image: 23
time taken by network : 4.602
image: 24
time tak

KeyboardInterrupt: 